# Exploration4 . Text Generation. 작사가 만들기


작사가는 노래의 가사를 전문적으로 만들어내는 사람이다.


Text Generation 을 활용하여 작사를 할 수 있을까


![image](https://user-images.githubusercontent.com/84179578/126740374-78072ca8-87a0-4667-90b2-b0a8fa1801e0.png)

## 1. 데이터 불러오기


여러 가사들에 대한 데이터는 `EP04_data` 파일에 `lyrics` 파일내에 저장되어있다. 파일을 읽어서 가사 데이터를 불러오자.

`glob` 모듈을 활용하면 파일을 읽어오는 작업을 하기가 편리하다.  
`glob` 을 포함하여 이번 프로젝트에서 사용할 여러 모듈들을 불러오자.

In [1]:
import glob
import re
import numpy as np
import tensorflow as tf

txt_file_path = 'EP04_data/lyrics/*'

txt_list = glob.glob(txt_file_path)

불러온 모든 `txt` 파일을 `raw_corpus` 리스트에 문장 단위로 저장한다.

In [2]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['[:Nicki Minaj]', 'Young money [Verse 1: Jason Derulo] (?) thousand different favors', 'I wish that I could (?)', "No i ain't got no dinner plans", 'So you should bring all your friends', 'I swear to (?) that you my type Noooo [Hook :Jason Derulo]', 'Shimmy shimmy she, shimmy she, shimmy yah', 'Swalla-la-la, swalla-la-la, swalla-la-la', 'Shimmy shimmy she, shimmy she, shimmy yah', 'Swalla-la-la, swalla-la-la, swalla-la-la']


## 2. 데이터 정제

우리가 불러온 문장 중 필요한 문장은 __가사__ 부분이다. 
따라서 다음과 같은 필요하지않은 부분은 제외한다.
- 공백인 문장
- 가수 파트 구분을 위해 대괄호가 포함된 부분
  - `'Young money [Verse 1: Jason Derulo] (?) thousand different favors'` 와 같이 파트 구분이 되고 앞뒤 내용이 이어지지 않는 문장이 많음
- 중복되는 문장


위 부분은 제외한 문장을 새로운 리스트 `corpus` 에 저장한다.

In [3]:
corpus= []

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue           # 길이가 0인 문장 제외
    if ('[' and ']') in sentence: continue    # 대괄호가 들어가는 문장 제외
    
    corpus.append(sentence)


print(f'원래 데이터 크기 : {len(raw_corpus)}')
print(f'공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : {len(corpus)}')

corpus = list(set(corpus))          # 중복불가한 set의 특성을 이용하여 중복되는 문장 제외 후 다시 리스트로 변환
print(f'중복되는 문장 제외한 데이터 크기 : {len(corpus)}')

원래 데이터 크기 : 187088
공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : 174446
중복되는 문장 제외한 데이터 크기 : 117114


공백인 문장, 대괄호를 포함한 문장, 중복된 문장이 제외된 것을 확인 할 수 있다.  그 과정을 완료한 후 데이터 수는 117114 개 이다.

## 3.토큰화

다음과 같은 순서로 문장을 일정한 기준으로 쪼개는 토큰화 (Tokenize) 를 진행한다.  
- 1. 소문자로 바꾸고, 양쪽 공백을 지움
- 2. 특수문자 양쪽에 공백을 넣음
- 3. 여러개의 공백은 하나의 공백으로 바꿈
- 4. `a-zA-Z?.!,¿`가 아닌 모든 문자를 하나의 공백으로 바꿈
- 5. 다시 양쪽 공백을 지움
- 6. 문장 시작에는 `<start>`, 끝에는 `<end>`를 추가

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)   # 4
    sentence = sentence.strip()                             # 5
    sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence

In [5]:
corpus_tokenized = []

for sentence in corpus:
    
    
    # 데이터 정제
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_tokenized.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus_tokenized[:10]

['<start> shoo be doo wa wa , dead on it <end>',
 '<start> that girl i can t take her <end>',
 '<start> you don t even gotta tell em <end>',
 '<start> go rimbaud go rimbaud go rimbaud <end>',
 '<start> the girl that now is dead <end>',
 '<start> i can live , i can breathe <end>',
 '<start> red white and blue , <end>',
 '<start> and i know they love it . <end>',
 '<start> yeah ! make that change ! <end>',
 '<start> soaked in soul <end>']

토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용한다.  

15000단어를 기억할 수 있는 tokenizer 생성한다. 그에 포함되지 않는 단어들은 `<unk>` 으로 나타낸다.  

또한, 지나치게 긴 문장들은 다른 데이터들이 과도한 padding 을 가지게 하므로 토큰의 개수가 15개를 초과하는 문장은 제외한다.

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,         # 15000개 단어를 기억할 수 있음
        filters=' ',
        oov_token="<unk>"        # 포함되지 않는 단어는 <unk> 으로 표현
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  # 토큰 15개 초과 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_tokenized)

[[   2 5169   27 ...    0    0    0]
 [   2   17   85 ...    0    0    0]
 [   2    7   35 ...    0    0    0]
 ...
 [   2 5413    6 ...    0    0    0]
 [   2  328   22 ...    0    0    0]
 [   2 1282  907 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f918c7d7cd0>


만들어진 tokenizer 을 보면 다음과 같다.

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


다음으로, tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다. 마지막 토큰은 `<end>`가 아니라 `<pad>`일 가능성이 높다.

tensor에서 `<start>`를 잘라내서 타겟 문장을 생성한다.

In [8]:
src_input = tensor[:, :-1] # 소스문장 생성 

tgt_input = tensor[:, 1:]  # 타켓 문장 생성

print(src_input[0])
print(tgt_input[0])

[   2 5169   27  958 2152 2152    4  405   20   11    3    0    0    0]
[5169   27  958 2152 2152    4  405   20   11    3    0    0    0    0]


## 4. 평가 데이터셋 분리

`sklearn.model_selection.trian_test_split` 을 이용해 training set 과 test set 을 분리한다.

이때 `test_size` 파라미터를 통해 test set 의 비율은 전체 데이터의 20% 로 지정한다.

In [9]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,       # 데이트셋 비율
                                                          shuffle=True, 
                                                          random_state=34)     # 결과를 일정하게 보여주기위해 지정

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (93691, 14)
Target Train: (93691, 14)


학습 데이터 개수가 93691 개인 것을 확인 할 수 있다.

## 5. 모델 학습

tf.keras.Model을 Subclassing하는 방식으로 모델을 만들 것이다.  

모델은 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어있다.

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

`embedding_size` 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기이다.
이번 예제에서는 `embedding_size` 를 256으로 지정하였다.   

`hidden_size` 는 LSTM 레이어의 hidden state 의 차원수이다. 
이번 예제에서는 `hidden_size` 는 2048 로 지정하였다.

In [12]:
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

epochs 값을 10으로 지정하여 모델을 학습시킨다.

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
2928/2928 [==============================] - 648s 213ms/step - loss: 3.7757
Epoch 2/10
2928/2928 [==============================] - 634s 217ms/step - loss: 3.0373
Epoch 3/10
2928/2928 [==============================] - 635s 217ms/step - loss: 2.7102
Epoch 4/10
2928/2928 [==============================] - 635s 217ms/step - loss: 2.3711
Epoch 5/10
2928/2928 [==============================] - 635s 217ms/step - loss: 2.0130
Epoch 6/10
2928/2928 [==============================] - 634s 217ms/step - loss: 1.6865
Epoch 7/10
2928/2928 [==============================] - 634s 216ms/step - loss: 1.4192
Epoch 8/10
2928/2928 [==============================] - 634s 216ms/step - loss: 1.2291
Epoch 9/10
2928/2928 [==============================] - 633s 216ms/step - loss: 1.1131
Epoch 10/10
2928/2928 [==============================] - 634s 216ms/step - loss: 1.0463


epoch 10번을 실행한 결과 loss가 1.0463 까지 떨어지는 것을 확인 할 수 있다.

## 6. 모델 평가

작문을 시켜보고 어떤지 직접 평가해보자.

In [14]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [18]:
generate_text(model, tokenizer, init_sentence="<start> girl", max_len=20)

'<start> girl , you got it <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a lie <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> if", max_len=20)

'<start> if you want to cross the bridge , my sweet <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i was", max_len=20)

'<start> i was born to make you happy <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are the greatest thing to me . <end> '

위와 같이 여러 단어를 통해 모델을 실행시킨 결과 실제 가사에 나올 법한 __sweeeeet__ 한 가사들이 생성되었다.

## 프로젝트 정리

많은 가사들을 불러와 데이터를 전처리하였다.   
- 가수 파트를 구별하는 대괄호가 포함된 문장 삭제
- 중복되는 문장 삭제
- 15000개의 단어를 기억할 수 있는 tokenizer 생성
- 토큰화 후 토큰 개수 15개 이상인 문장 삭제

그것을 이용해 모델을 학습시켜 가사를 생성해내는 모델을 만들었다.   

파라미터 값을 데이터에 알맞게 설정하여 모델을 학습시켰다.

학습시킨 모델이 `i was born to make you happy`, `you are the greatest thing to me` 와 같이 굉장히 감성적인 가사를 쓸 수 있는 것을 확인하였다.  

하지만 여전히 문장 문법이 어색하거나, 내용이 어색한 문장도 종종 생성되었다.  

학습시키는 데이터 수를 늘리면 이러한 문제가 줄어들고 점점 사람이 만들어내는 가사와 비슷한 가사들이 생성될 것이다.




이번 프로젝트를 통해 인공지능이 문장을 이해하는 방식에 대해 알아볼 수 있었고 어떻게 모델을 학습시켜서 작문을 하게 하는지에 대한 전체적인 구조에 대해 이해할 수 있었다.